In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X = torch.rand(size=(2,4))
net(X)

tensor([[-0.1815],
        [-0.1418]], grad_fn=<AddmmBackward0>)

# 参数访问
通过 `Sequential` 类定义模型时，可以通过索引访问模型的任意层

In [4]:
# 打印第三层的参数字典
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.1104, -0.1236,  0.2629,  0.2068,  0.0386, -0.0805, -0.2181,  0.2489]])), ('bias', tensor([-0.2870]))])


## [目标参数]
对参数执行任何操作前，需要查看底层数值，方法如下：
1. 查看数据类型
2. 查看参数详细信息，包括值、梯度、额外信息
3. 查看参数数值

In [6]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2870], requires_grad=True)
tensor([-0.2870])


参数是复合的对象，包括值、梯度和额外信息，所以需要用 `.data` 的方法显式查看参数值

## [一次性访问所有参数]
逐个访问很麻烦，是按照树结构存储的，有特定的方法

In [10]:
# 逐个访问
print(*[(name,param.shape) for name,param in net[0].named_parameters()]) # 第一层网络的参数
print(*[(name,param.shape) for name,param in net.named_parameters()]) # 全部参数

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


## [访问嵌套块的参数]


In [14]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())
def block2():
    net = nn.Sequential()
    # 循环生成嵌套块
    for i in range(4):
        net.add_module(f"block {i}",block1())
    return net

rgnet = nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[-0.2553],
        [-0.2553]], grad_fn=<AddmmBackward0>)

查看网络是如何工作的

In [15]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


可以看到网络是分层嵌套的， `Sequential` 里面还套有 `Sequential` ，同样的，我们也可以通过下标进行访问

In [17]:
# 查看rgnet第1个主要块中第2个子块的第三层网络的偏置数值
print(rgnet[0][1][2].bias.data)

tensor([-0.0126, -0.1622, -0.2808, -0.0550])


# 参数初始化
在 `ch_4` 中，有提到参数初始化1的重要性。

常见的初始化如下：
- 内置初始化
- 自定义初始化

In [20]:
# 权重从正态分布中采样初始化
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([-0.0062,  0.0134, -0.0076,  0.0114]), tensor(0.))

In [22]:
# 给定值初始化
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [26]:
# Xavier初始化
def init_xavier(m):
     if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)
net.apply(init_xavier)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([0.2657, 0.3424, 0.6131, 0.0993]), tensor(0.))

In [31]:
# 自定义初始化
# w从 U(5,10)，0，U(-10,-5) 中采样的概率分别为 1/4 1/2 1/4
def my_init(m):
    if type(m) == nn.Linear:
        print("init",*[(name,param.shape) for name,param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10) # 使用 U(-10,10) 
        m.weight.data *= m.weight.data.abs() >= 5
    
net.apply(my_init)
net[0].weight[:2]

init weight torch.Size([8, 4])
init weight torch.Size([1, 8])


tensor([[-6.6915, -6.4184, -0.0000, -7.2238],
        [-0.0000,  0.0000,  0.0000, -7.7978]], grad_fn=<SliceBackward0>)

需要注意的是，我们一直可以设置参数

In [32]:
net[0].weight.data[:] += 2
net[0].weight.data[0,0] = 45
net[0].weight.data[0]

tensor([45.0000, -4.4184,  2.0000, -5.2238])

# 参数绑定
参数共享就是将指定层的参数进行绑定，不只是值相同，而是用同一个的对象表示层的参数

定义一个稠密层，然后就可以用它的参数来设置另一个层的参数

In [40]:
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                    shared,nn.ReLU(),
                    shared,nn.ReLU(),
                    nn.Linear(8,1))
net(X)

# 查看绑定的参数
# 值是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
# 是否绑定
net[2].weight.data[0,0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


说明第三层和第五层的参数是绑定的，不仅值相等，而且使用相同的张量表示。

当参数绑定时，由于模型参数包含梯度，所以在反向传播时第二层和第五层的梯度 **会加在一起** 。